# quantum_machine_learning.myQML.QCCNN

**class quantum_machine_learning.myQML.QCCNN(n_qubits, device, n_blocks, n_shots, optimizer_name, loss, learning_rate = 0.01, opt_model_path=None, np_arrays_path=None)**    

Class to solve an Supervised Machine Learning image multi-class classification problem using a Quantum-Classical Convolutional Neural Network. It defines a quantum convolutional layer and then a classical convolutional neural network. First, using random quantum variational angles, it processes the images and trains the classical network to find the optimal classical weights, and then, with these fixed weights, it trains the quantum layer. This algorithm is inspired by the paper *Quanvolutional Neural Networks: Powering image Recognition with Quantum Circuits* by Henderson et al. However, in the paper the angles of the quantum convolutional layer are fixed, while here we optimize them too. 

**Parameters:**
- **n_qubits** (int): Number of qubits used for the quantum circuit, which depends of the size of the quantum kernel used to process the image.
- **device** (str): Indicates whether the quantum circuits are executed locally (myQLM) or sent to a Qaptiva emulator as batches.
- **n_blocks** (int): Number of blocks in the variational quantum layer, where each of the blocks contains rotational gates and CNOTs (or other entangling two-qubit gates).
- **n_shots** (int): Number of executions of the corresponding quantum circuit to estimate a probability or the expectation value of an observable. If n_shots = None, the estimation of the quantum simulator has no shot noise.
- **optimizer_name** (str): Label of the optimizer used for the training of the classical neural network. Only 'Adam' optimizer is implemented currently.
- **loss** (str): Label of the loss function used for the training of the classical neural network, e.g. 'sparse_categorical_crossentropy'. Any loss function label already implemented in tensorflow keras can be used.
- **learning_rate** (str): Size of the step in the Adam optimizer. Not needed if a different optimizer is used.
- **opt_model_path** (str): Path where the optimal classical convolutional neural network is going to be saved.
- **np_arrays_path** (str): Path where the images after the quantum processing are saved, as well as the quantum random angles used for that processing.


**quantum_conv_kernel_circuit(x)**

This method implements a quantum convolutional kernel circuit for a custom QCCNN. It first applies an angle encoding feature map to map the input data (portion of the image) to a quantum state, and then applies series of rotation gates with random angles and CNOT gates to the qubits.

**Parameters:**
- **x** (numpy.ndarray): Input data vector of length n_qubits that is used as input angles of the rotation gates applied to each qubit in the feature map. 

**Returns**: An array of n_qubits elements with the Z expectation value of each of the qubits.

**Return type**:  numpy.ndarray


**quantum_conv_layer(image)**

This method slides the quantum kernel circuit through the whole images and returns the corresponding output image.


**Parameters:**
- **image** (numpy.ndarray): Rank 2 or 3 tensor with the pixel values of the image (with possibly more than one channel). 

**Returns:** The resulting image or tensor of usually rank 3.

**Return type:** numpy.ndarray 


**quantum_conv_preprocessing(train_images, test_images=None, save=True, params=None)**

This method calls quantum_conv_layer for all the images in the training and test sets, and stack the resulting tensor forming a rank 4 tensor for each set.

**Parameters:**
- **train_images** (numpy.ndarray): Images (with possibly multiple channels) in the training set used to optimize the network.
- **test_images** (numpy.ndarray): Images (with possibly multiple channels) in the test set used to assess the performance of the network.
- **save** (bool): Determines whether the train and test output quantum images are saved.
- **params** (numpy.ndarray): Array of the random angles of the variational ansatz of the quantum convolutional layer. If None, a new array of andom angle is created.

**Returns:** The output tensors resulting from the quantum preprocessing of the training and test sets, i.e., the quantum_train_images and quantum_test_images. It also returns the random angles employed.

**Return type:** quantum_train_images - numpy.ndarray (4D), quantum_test_images - numpy.ndarray (4D), params - numpy.ndarray (1D).


**classical_model()**

Defines a classical CNN model (using tensorflow keras) with custom hidden layers with the desired filters, kernel size and ativation function, as well as the output layer with the dsired number of classes and activation function. Additionally, it specifies the optimizer, loss function and metrics (such as the accuracy).

**Returns:** The compiled tensorflow keras model.



**train(preprocessing, train_images, train_labels, validation_images, validation_labels, batch_size, n_epochs, early_stop_crit=False, patience=None, min_delta=None)**

This method trains the QCCNN model using the quantum convolutional preprocessing images as input, optimizing the classical CNN to minimize the training loss.

**Parameters:**
- **preprocessing** (bool): Indicates whether quantum preproccsing is required, in case the input images have arleady been processed with the quantum layer.
- **train_images** (numpy.ndarray): Images (with possibly multiple channels) in the training set used to optimize the network.
- **train_labels** (numpy.ndarray): Array containing the labels of each of the training images.
- **validation_images** (numpy.ndarray): Images (with possibly multiple channels) not used for training but for monitoring the performance of the training.
- **validation_labels** (numpy.ndarray): Labels of the validation images.
- **batch_size** (int): The number of training images in a batch used to calculate the average loss function and its gradient to update the network weights and biases, typically used in stochastic gradient descent based methods.
- **n_epochs** (int): Number of times the whole training set is utilized to update the network weights and biases. In each of the epochs, different batches are formed, and used to evaluate the loss function and gradient to optimize the neural network
- **early_stop_crit** (bool): Determines whether an early stop criterium based on validation images is used to prevent overfitting
- **patience** (int): Number of epochs to continue training after the last time the monitored performance metric improved. If the metric does not improve within this number of epochs, training stops early.
- **min_delta** (float): Minimum change in the monitored metric (validation accuracy) that classifies as mprovement. Changes smaller than min_delta are considered insignificant and do not reset the patience counter.

**Returns:** The training history, including the training and validation accuracy and loss at each epoch.


**plot_quantum_images(train_images, quantum_train_images, n_samples)**

Plot n_samples examples of the output images resulting from the quantum convolutional layer preprocessing.

**Parameters:**
- **train_images** (numpy.ndarray): Images (with possibly multiple channels) in the training set used to optimize the network. This multidimensional array is a rank 3 tensor if there is only one channel and a rank 4 tensor if there are more than one (rgb colors).
- **quantum_train_images** (numpy.ndarray): Images after applying the quantum convolutional preprocessing. Each quantum output image is a rank 3 tensor, so the set of all the images is a rank 4 tensor.
- **n_samples**: Number of example images plotted.

**Returns:** This method does not return any variable, but shows the figure in the notebook/terminal and saves it.



**plot_loss(c_history=None, fig_path=None, fig_name='qccnn_loss.png')**

Plots two figures: one for the evolution of the training and validation accuracy of the model with and without the quantum convolutional layer, and the same figure but for the loss.

**Parameters:**
- **c_history** (keras.callbacks.History): Training and validation accuracy, and training and validation loss of the netwrok in each epochs.
- **fig_path** (str): Path where the figure is saved.
- **fig_name** (str): Name of the saved image.

**Returns:**  This method does not return any variable, but shows the figure in the notebook/terminal and saves it.



**optimize_quantum_params(train_images, train_labels, method='cobyla', max_iter=10, n_init=5, n_iters=20)**

Fixing the optimal weights and biases of the classical CNN, we vary the angles of the rotational gates in the quantum layer to minimize the training loss, using different training data. We implement wo different gradient-free optimization methods: Cobyla and Bayesian Optimization. In the future, parameter shift rule or autodifferentiation could be implemented.

**Parameters:**
- **train_images** (numpy.ndarray): Images (with possibly multiple channels) in the training set used to optimize the network. This multidimensional array is a rank 3 tensor if there is only one channel and a rank 4 tensor if there are more than one (rgb colors).
- **train_labels** (numpy.ndarray): Array containing the labels of each of the training images.
- **method** (str): Can be 'cobyla' or 'bayesian_optimization'
- **max_iter** (int): Maximum number of iterations (for cobyla)
- **n_init** (int): Initial number of evalutations at random parameters within the bounds.
- **n_iters** (int): Number of iterations of bayesian optimization.

**Returns:** The history of the optimization (training loss at each epoch), an array wth the training accuracies, and an array with the training losses.



**predict(preprocessing, test_images, test_labels)**

Evaluates the performance of the quantum enhanced convolutional network on the test set.

**Parameters:**
- **preprocessing** (bool): Indicates whether it is necessary to do the quantum preprocessing on the test images. If an optimization of the angles in the quantum layer has been performed, then, preprocessing with those optimized angles is required.
- **test_images** (numpy.ndarray): Images (with possibly multiple channels) not used for training but for assessing the performance of the network at classifying the images.
- **test_labels** (numpy.ndarray): Labels of the test images.

**Returns:** The test loss and test accuracy (both float variables).



**plot_q_train_loss(train_accuracies, train_losses, fig_path=None, fig_name='qccnn_quantum_loss.png')**

Plots the training accuracy during the optimization of the angles in the quantum convolutional layer.

**Parameters:**
- **train_accuracies** (numpy.ndarray): Array with the training accuracy in each optimization step. 
- **train_losses** (numpy.narray): Array with the train loss in each optimization step.
- **fig_path**: Path where this figure is saved.
- **fig_name**: Name of the figure saved as an image.

**Returns:** This method does not return any variable, but shows the figure in the notebook/terminal and saves it.
